# AgentCore Observability

In this tutorial we will showcase CloudWatch GenAI Observability Dashboard. We will run a few tests and then review the traces generated by our agent.

Amazon Bedrock AgentCore Observability helps you trace, debug, and monitor agent performance in production environments.

AgentCore Observability provides:

* Detailed visualizations of each step in the agent workflow
* Real-time visibility into operational performance through CloudWatch dashboards
* Telemetry for key metrics such as session count, latency, duration, token usage, and error rates
* Rich metadata tagging and filtering for issue investigation
* Standardized OpenTelemetry (OTEL)-compatible format for easy integration with existing monitoring stacks


<div style="text-align:left">
    <img src="images/00-Observability.png" width="75%"/>
</div>

### Observability Concepts in AgentCore
This section defines the concepts of sessions, traces and spans as they relate to monitoring and observability of agents.
        
##### Sessions
A session represents a complete **interaction context** between user and agent. Sessions encapsulate the entire conversation or interaction flow, maintaining state and context across multiple exchanges. Each session has a unique identifier and captures the full lifecycle of user engagement with the agent, from initialization to termination.
##### Traces
A trace represents a detailed record of a single request-response cycle beginning from with an agent invocation and may include additional calls to other agents. Traces capture the **complete execution path** of a request, including all internal processing steps, external service calls, decision points, and resource utilization. 
##### Spans
A span represents a discrete, measurable **unit of work** within an agent's execution flow. Spans capture fine-grained operations that occur during request processing, providing detailed visibility into the internal components and steps that make up a complete trace. Each span has a defined start and end time, creating a precise timeline of agent activities and their durations.


The relationship between these three observability components can be visualized as:
- Sessions (highest level) - Represent complete user conversations or interaction contexts
- Traces (middle level) - Represent individual request-response cycles within a session
- Spans (lowest level) - Represent specific operations or steps within a trace


### Step 1: Examining how OpenTelemetry is configured with AgentCore

To enable tracing of AI applications, we need to add the AWS Distro for Open Telemetry (ADOT) SDK  `aws-opentelemetry-distro` to your agent code. When using the `bedrock_agentcore_starter_toolkit` to configure your agent, it takes care of the opentelemetry instrumentation for you.

More details are available [here](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-configure.html#observability-configure-custom).

1. Open the Code server. Look for the deploy_agentcore/Dockerfile that was created as part of the deployment process in the previous lab.

2. Observe that it installs OpenTelemetry for you with RUN pip install aws-opentelemetry-distro>=0.10.1 and then runs it as part of the deployment process CMD ["opentelemetry-instrument", "python", "-m", "xxx"].

### Step 2: Enable CloudWatch Transaction Search (One-Time Setup)

**Note:** You can skip this step if lab 3.2 was executed. You will not see the "Enable" button if Transaction Search is already enabled.

This is a one-time setup to turn on Amazon CloudWatch Transaction Search. Follow the steps below to enable:

1. Open the [CloudWatch Console](https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1)
2. Navigate to **Application Signals (APM)** → **Transaction search**
3. Choose **Enable Transaction Search**
4. Select checkbox to **ingest spans as structured logs**
5. (Optional) Adjust **X-Ray trace indexing** percentage (default: 1%)
6. Choose **Save**

<div style="text-align:left">
    <img src="images/observability-ts.png" width="75%"/>
    <img src="images/observability-ts-save.png" width="75%"/>
</div>

### Step 3: Run a few invocations

When using ADOT, in order to propagate session id correctly, define the X-Amzn-Bedrock-AgentCore-Runtime-Session-Id in the request header. ADOT then sets the session_id correctly in the downstream headers.

To propagate a trace ID, invoke the AgentCore runtime with the parameter traceId=<traceId> set.

We can invoke the agent using either the AWS CLI or boto3 (AWS Python SDK).

The AWS CLI has both Control Plane (e.g., Create/Update/List/Describe Agents/Memory Strategies) and Runtime (e.g., Invoke Agent/Read Memory/Stop Agent) commands.

Let's explore both approaches as we generate telemetry data for observability.

#### Invoke Your Agent Using AWS CLI

In [ ]:
# List available runtimes to verify your agent is deployed and get its ARN
!aws bedrock-agentcore-control list-agent-runtimes

**Invoking AgentCore Runtime with AWS CLI**

After listing the available runtimes, you can invoke your agent directly using the AWS CLI. This is useful for testing, automation, or integration with other systems.

First, create a payload file with your request:

In [ ]:
%%writefile payload.json
{"query": "What products have the best reviews?"}

Now invoke the agent runtime and view the response:

**Key parameters:**
- `--agent-runtime-arn`: The ARN of your deployed agent (obtained from previous steps)
- `--payload`: Input data as a file or blob (JSON format for most agents)
- `--content-type`: MIME type of the payload (typically "application/json")
  `--cli-binary-format raw-in-base64-out: Explicitly set the output format \
- `response.json`: Output file where the response will be saved

**Optional parameters for advanced use:**
- `--runtime-session-id`: Maintain conversation context across multiple invocations
- `--runtime-user-id`: Track invocations by specific users
- `--qualifier`: Target a specific version or endpoint of the agent

For more details, see the [AWS CLI invoke-agent-runtime documentation](https://docs.aws.amazon.com/cli/latest/reference/bedrock-agentcore/invoke-agent-runtime.html).

In [ ]:
# Invoke the agent runtime (replace <YOUR_AGENT_ARN> with your actual agent ARN)
# The --cli-binary-format allows us to use the json directly.  Without this, you need to base64 encode the payload
!aws bedrock-agentcore invoke-agent-runtime \
  --agent-runtime-arn "<YOUR_AGENT_ARN>" \
  --payload file://payload.json \
  --content-type "application/json" \
  --cli-binary-format raw-in-base64-out \
  response.json

# View the response
!jq . response.json

In [ ]:
# Invoke the agent runtime (replace <YOUR_AGENT_ARN> with your actual agent ARN)
!aws bedrock-agentcore invoke-agent-runtime \
  --agent-runtime-arn "<YOUR_AGENT_ARN>" \
  --payload '{"query": "How many products do we have?" }' \
  --content-type "application/json" \
  --cli-binary-format raw-in-base64-out \
  response.json

# View the response
!jq . response.json

#### Invoke Your Agent Using boto3

Let's import the main libraries that we need, and initialize some common variables used further down - `account_id`, `region` and `agentcore_runtime`

In [ ]:
import json
import os
import random, string
import time

import boto3
from bedrock_agentcore_starter_toolkit import Runtime

boto_session = boto3.Session()
region = boto_session.region_name

print(f"current region: {region}")
account_id = boto_session.client("sts").get_caller_identity()["Account"]
print(f"current account: {account_id}")

agentcore_runtime = Runtime()

The next cell will automatically retrieve the agent ARN from the previous labs or the AgentCore Runtime

In [ ]:
# Get the agent ARN from stored variable or AgentCore Runtime
%store -r agent_arn

if 'agent_arn' not in locals():
    try:
        status_response = agentcore_runtime.status()
        agent_arn = status_response['endpoint']['agentRuntimeArn']
        print(f"Retrieved agent ARN: {agent_arn}")
    except Exception as e:
        print(f"Error: {e}")
        print("Please run Lab 3.1 or 3.2 first to deploy the agent.")
        agent_arn = "<REPLACE_WITH_YOUR_AGENT_ARN>"
else:
    print(f"Using stored agent ARN: {agent_arn}")

Define a helper function to invoke the agent with proper session tracking for observability:

In [ ]:
def ask(question, user_id="user1", session_id=None):
    if session_id is None:
        session_id = f"session_{user_id}"
    
    payload = json.dumps({"prompt": question, "user_id": user_id})
    
    response = agentcore_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        qualifier="DEFAULT",
        payload=payload,
        runtimeSessionId=session_id
    )
    
    # Read and parse response
    body = response['response'].read().decode('utf-8')
    data = json.loads(body)
    
    # Extract and print the text content (which will render \n as newlines)
    text = data['result']['content'][0]['text']
    
    print(f"Q: {question}\n")
    print(text)
    print("\n" + "="*80 + "\n")

Initialize the boto3 client and set up session tracking to ensure all invocations are grouped together in CloudWatch for easier analysis:

In [ ]:
import json

import boto3
from IPython.display import Markdown, display

agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# Set a user id and session id for our test
user_id = "user1"
session_id = f"test_observability_session_for_{user_id}"

Now invoke the agent to generate telemetry data:

In [ ]:
ask(
    "How many customers reviewed product_890, are those reviews positive or negative?",
    user_id=user_id,
    session_id=session_id
)

### Step 4: GenAI Observability for visualization

The [CloudWatch GenAI Observability](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GenAI-observability.html) provides Bedrock AgentCore dashboard with Agent, Sessions View & Traces Views to understand the performance and execution flow of Runtime-hosted agents. You can access them by selecting GenAI Observability (Preview) in the CloudWatch console.

CloudWatch GenAI Observability provides two pre-built dashboards:
- Model Invocations – Detailed metrics on model usage, token consumption, and costs
- Amazon Bedrock AgentCore agents – Performance and decision metrics for the Amazon Bedrock agents

Click [here](https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#/gen-ai-observability/agent-core?start=-3600000) to navigate to the Bedrock AgentCore dashboard. You can analyze various Agents and their associated interactions under Agent view, Sessions view, and Traces view.

Click on **Sessions view** and you will be able to see the session ID that we just tested with above.

<div style="text-align:left">
    <img src="images/02-Sessions-view.jpg" width="50%"/>
</div>

### Understanding the Three Dashboard Views

The CloudWatch GenAI Observability dashboard provides three complementary views:

**1. Agent View (High-Level Metrics)**
- Overall agent performance across all sessions
- Invocation counts and success rates
- Error rates and latency percentiles
- Useful for: Monitoring overall agent health

**2. Sessions View (Conversation-Level)**
- Individual conversation threads
- Session duration and message counts
- User interaction patterns
- Useful for: Understanding user engagement and conversation flows

**3. Traces View (Execution-Level)**
- Detailed execution paths for each invocation
- LLM calls with token usage
- Tool invocations and timing
- Knowledge base retrievals
- Useful for: Debugging issues and optimizing performance

**Trace Trajectory:**
When you select a trace, the "Trajectory" view shows the complete execution flow:
- Which tools were called and in what order
- How long each operation took
- What data was passed between components
- Where errors occurred (if any)

This visibility is essential for understanding agent behavior and troubleshooting issues in production.

Select session "test_observability_session_700001" to see a list of traces for this session. In our test there is only trace.

The trace provides end-to-end visibility into agent execution paths including LLM calls and tool usage. Select "Trajectory" to review the interconnected relationship of the spans and subsequent calls from these spans. For illustration, the screenshot below highlights execution of one of the tools available to the agent and its execution time.

<div style="text-align:left">
    <img src="images/03-trace-view.jpg" width="70%"/>
</div>


### View Logs in CloudWatch

1. Open the [CloudWatch console](https://us-east-1.console.aws.amazon.com/cloudwatch/)
2. In the left navigation pane, expand **Logs** and select **Log groups**
3. Search for your agent's log group:
   - Standard logs (stdout/stderr): `/aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/[runtime-logs] <UUID>`
   - OTEL structured logs: `/aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/runtime-logs`

### CloudWatch Logs Structure

AgentCore automatically creates two types of log groups for your agent:

**Standard Runtime Logs:**
- Path: `/aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/[runtime-logs]`
- Contains: Application logs, print statements, error messages
- Use for: General debugging and application-level troubleshooting

**OpenTelemetry Structured Logs:**
- Path: `/aws/bedrock-agentcore/runtimes/<agent_id>-<endpoint_name>/runtime-logs`
- Contains: Detailed telemetry data, spans, traces
- Use for: Performance analysis and detailed execution tracing

Both log groups are automatically created when you deploy your agent. No additional configuration needed!

### View Metrics

1. Open the [CloudWatch console](https://us-east-1.console.aws.amazon.com/cloudwatch/)
2. Select **Metrics** from the left navigation
3. Browse to the `bedrock-agentcore` namespace
4. Explore the available metrics

See [here](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability-service-provided.html) for more details on available metrics.

### Available CloudWatch Metrics

AgentCore publishes metrics to the `bedrock-agentcore` namespace:

**Invocation Metrics:**
- `Invocations`: Total number of agent invocations
- `Errors`: Number of failed invocations
- `Duration`: Execution time per invocation

**Resource Metrics:**
- `CPUUtilization`: Container CPU usage
- `MemoryUtilization`: Container memory usage

**Custom Metrics:**
You can also publish custom metrics from your agent code using CloudWatch PutMetricData API.

**Setting Up Alarms:**
Use these metrics to create CloudWatch alarms for:
- High error rates
- Increased latency
- Resource exhaustion
- Unusual invocation patterns

This enables proactive monitoring and alerting for your production agents.

### Summary
In this tutorial, we demonstrated how to enable CloudWatch GenAI Observability for AgentCore and how to view traces and sessions for a test agent. We also reviewed the available metrics and logs for further analysis and monitoring of the agent's performance. This enables us to gain insights into the agent's performance, identify potential issues, and improve its overall operational efficiency


## Next Steps

Congratulations! You've completed all workshop labs and now have a production-ready intelligent RAG agent with:
- Unstructured and structured knowledge bases
- AgentCore Runtime deployment with automatic scaling
- Memory capabilities for conversation context
- Full observability through CloudWatch

**Cleanup:**
- **Workshop-supplied environment:** Simply close your browser tabs - resources will be cleaned up automatically
- **Your own AWS account:** Follow the [cleanup instructions](../Cleanup-Instructions.ipynb) to delete resources and avoid ongoing charges
